In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 21 10:06:47 2020

@author: williammartin
"""

# import standard libraries
import numpy as np
# import third-party libraries
import matplotlib.pyplot as plt
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import LinearRegression
import wrds
# import local libraries

plt.close('all')
DOWNLOAD = False
LOAD_CACHE = True

class Stock:
    
    def __init__(self, permno, df):
        self.permno = permno
        self.df = df
        # compute market capitalization
        self.df['mcap'] = self.df['shrout'] * self.df['prc'].abs()
        
    def computeAverageReturn(self):
        """
        For each stock and each month t, calculate the average 
        return of the stock between month t-12 and t-2.
        
        We assume there are no jumps of months and apply 
        the average over the integer index and not the datetime.
        """
        ret_lag = self.df[['ret']].shift(2)
        mean_ret = ret_lag.rolling(10+1).mean()
        mean_ret = mean_ret.rename(columns = {'ret': 'mean_ret'})
        self.df = pd.concat([self.df, mean_ret], axis = 1)
        
        
if __name__ == '__main__':
    
    start = '1970-01-01'
    end = '2019-12-31'

    if DOWNLOAD:
        
        # connect to db
        db = wrds.Connection(wrds_username = 'wmartin')
        # db.create_pgpass_file()

        # get risk free
        rf = db.raw_sql("select mcaldt,tmytm "
                        "from crsp.tfz_mth_rf "           
                        "where kytreasnox = 2000001 "
                        "and mcaldt>='{}'"
                        "and mcaldt<='{}'".format(start, end), 
                        date_cols = ['mcaldt'])
        
        # get crsp value-weighted idnex return
        crsp_index = db.raw_sql("select date,vwretd "
                                "from crsp.msi "
                                "where date>='{}'"
                                "and date<='{}'".format(start, end), 
                                date_cols = ['date'])
        
        # get crsp stock event
        crsp_stock = db.raw_sql("select a.permno, a.date, "
                                "b.shrcd, b.exchcd, a.ret, a.shrout, a.prc "
                                "from crsp.msf as a "
                                "left join crsp.msenames as b " 
                                "on a.permno=b.permno " 
                                "and b.namedt<=a.date " 
                                "and a.date<=b.nameendt "
                                "where a.date between '{}' and '{}' "
                                "and b.exchcd in (1, 2) "
                                "and b.shrcd in (10, 11) ".format(start, end), 
                                date_cols=['date']) 
        
        crsp_stock = crsp_stock.drop(columns = ['shrcd', 'exchcd'])
        
        # write to csv raw
        rf.to_csv('rf.csv')
        crsp_index.to_csv('crsp_index.csv')
        crsp_stock.to_csv('crsp_stock.csv')

        
    else:
        
        # read locally
        rf = pd.read_csv('rf.csv', usecols = [1, 2])
        rf['mcaldt'] = pd.to_datetime(rf['mcaldt'])
        
        crsp_index = pd.read_csv('crsp_index.csv', usecols = [1, 2])
        crsp_index['date'] = pd.to_datetime(crsp_index['date'])
        
        crsp_stock = pd.read_csv('crsp_stock.csv', usecols = [1, 2, 3, 4, 5])
        crsp_stock['date'] = pd.to_datetime(crsp_stock['date'])
    
    # clean rf
    rf = rf.rename(columns = {'mcaldt': 'date', 'tmytm': 'rf'})
    
    # clean data a bit
    rf = rf.set_index('date')
    crsp_index = crsp_index.set_index('date')
    crsp_stock = crsp_stock.set_index('date')
    
    # floatify data
    crsp_stock['permno'] = \
        crsp_stock['permno'].astype(int)
        
    # correct rf to monhtly returns
    rf = rf.applymap(lambda x: np.exp(x/12/100) - 1)
    
    # read fama data
    mom = pd.read_csv('F-F_Momentum_Factor.csv', skiprows = 12, index_col = 0)
    mom = mom.loc[:'201912']
    mom = mom.rename(columns = {'Mom   ': 'Mom'})
    fama = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', skiprows = 3, index_col = 0)
    fama = fama.loc[:'201912']
    # set index of fama website to datetime (end of month)
    mom.index = (pd.to_datetime(mom.index, format = '%Y%m')+MonthEnd(1))
    fama.index = (pd.to_datetime(fama.index, format = '%Y%m')+MonthEnd(1))
    # filter date from mom and fama
    mom = mom.loc[start:end, :]
    fama = fama.loc[start:end, :]
    
    # change columns types
    mom = mom.astype(float)
    fama = fama.astype(float)
    
    # divide fama and mom by 100
    fama /= 100
    mom /= 100
    
    # split crsp_stock by stock
    crsp_stock_grouped = crsp_stock.groupby('permno')
    # create stocks from data
    stocks = {}
    for permno, df in crsp_stock_grouped:
        df = df.drop(columns = ['permno'])
        df = df.sort_index()
        # do a forward fill and backward fill if missing values
        # df = df.dropna(how = 'any', axis = 0)
        #df = df.fillna(method = 'ffill')
        #df = df.fillna(method = 'bfill')
        stocks[permno] = Stock(permno, df)
        
        
# =============================================================================
# (b)
# =============================================================================
        
    if not LOAD_CACHE: 
    
        # compute average returns as written in the exercise sheet
        for _, stock in stocks.items():
            stock.computeAverageReturn()
            
        # create a dataframe containing the average returns of all stocks
        mean_ret_stocks = pd.DataFrame()
        for permno, stock in stocks.items():
            temp = stock.df[['mean_ret']].rename(columns = {'mean_ret': permno})
            mean_ret_stocks = pd.concat([mean_ret_stocks, temp], axis = 1)
            
        mean_ret_stocks.to_csv('mean_ret_stocks.csv')
        
        # construct dataframe of all returns of all stocks
        ret_stocks = pd.DataFrame()
        for permno, stock in stocks.items():
            temp = stock.df[['ret']].rename(columns = {'ret': permno})
            ret_stocks = pd.concat([ret_stocks, temp], axis = 1)
            
        ret_stocks.to_csv('ret_stocks.csv')
        
        # construct dataframe of all market caps
        mcap_stocks = pd.DataFrame()
        for permno, stock in stocks.items():
            temp = stock.df[['mcap']].rename(columns = {'mcap': permno})
            mcap_stocks = pd.concat([mcap_stocks, temp], axis = 1)
            
        mcap_stocks.to_csv('mcap_stocks.csv')
        
    else: # faster if saved in csv
        
        mean_ret_stocks = pd.read_csv('mean_ret_stocks.csv', index_col = 'date')
        mean_ret_stocks.index = pd.to_datetime(mean_ret_stocks.index)
        mean_ret_stocks.columns = mean_ret_stocks.columns.astype(int)

        # put mean_ret back in stocks
        for permno in mean_ret_stocks:
            stocks[permno].df['mean_ret'] = mean_ret_stocks[permno]
            
        ret_stocks = pd.read_csv('ret_stocks.csv', index_col = 'date')
        ret_stocks.index = pd.to_datetime(ret_stocks.index)
        ret_stocks.columns = ret_stocks.columns.astype(int)
        
        mcap_stocks = pd.read_csv('mcap_stocks.csv', index_col = 'date')
        mcap_stocks.index = pd.to_datetime(mcap_stocks.index)
        mcap_stocks.columns = mcap_stocks.columns.astype(int)
            
    # drop months where there is nothng (due to shift and rolling methods)
    mean_ret_stocks = mean_ret_stocks.dropna(how = 'all', axis = 0)
    # in case
    ret_stocks = ret_stocks.dropna(how = 'all', axis = 0)
    mcap_stocks = mcap_stocks.dropna(how = 'all', axis = 0)
        
    # in each month, sort the stocks into 10 deciles based on that average return
    # we omit stocks that were not traded when going through months
    # duplicates = False
    mean_ret_deciles = mean_ret_stocks.apply(lambda x: 1 + pd.qcut(x, 
                                                                   10, 
                                                                   labels = False,
                                                                   duplicates = 'drop'), 
                                             axis = 1)
    
    # a quick check using value counts over each row tells us that 
    # the binning was done correctly
    
    # truncate ret_stocks
    ret_stocks = ret_stocks.iloc[12:]
    mcap_stocks = mcap_stocks.iloc[12:]
    
    # create value-weighted returns every month based on 10 deciles
    vw_returns = pd.DataFrame(index = mean_ret_deciles.index,
                              columns = list(range(1, 11)))
    
    # fill vw_returns
    # go over all bins
    for b in vw_returns.columns:
        returns_b = ret_stocks.mask(mean_ret_deciles != float(b), np.nan)
        mcaps_b = mcap_stocks.mask(mean_ret_deciles != float(b), np.nan)
        returns_b = returns_b.dropna(how = 'all')
        mcaps_b = mcaps_b.dropna(how = 'all')        
        weights_b = mcaps_b.apply(lambda row: row/row.sum(), axis = 1)
        vw_returns_b = pd.DataFrame((weights_b*returns_b)\
                                    .sum(axis = 1)/weights_b.sum(axis = 1))
        vw_returns[b] = vw_returns_b
        
    # Compute the returns on a zero-cost portfolio that 
    # goes long in the group with the highest past returns 
    # and short in the group with the lowest past returns
    zero_cost_returns = vw_returns[10] - vw_returns[1]
    
    # get expected zero cost returns
    rf_fama = fama.iloc[12:]['RF']

    #Compute the alpha of this strategy with respect to the SMB and HML factors.
    # get same index in fama as in zero_cost_returns
    smb_hml = fama.iloc[12:][['Mkt-RF', 'SMB', 'HML']]
    smb_hml_mom = pd.concat([smb_hml,  mom.iloc[12:]], axis = 1) 
    
    # do a linear regression
    regr_fama = LinearRegression()
    regr_fama = regr_fama.fit(smb_hml, zero_cost_returns)
    alpha_fama = regr_fama.intercept_
    beta_fama = regr_fama.coef_
    
    regr_fama_mom = LinearRegression()
    regr_fama_mom = regr_fama_mom.fit(smb_hml_mom, zero_cost_returns)
    alpha_fama_mom = regr_fama_mom.intercept_
    beta_fama_mom = regr_fama_mom.coef_
    
    # alpha_fama = 0.009851730287095881
    # alpha_fama_mom = -0.0037760959322449853
    # beta_fama = array([-0.38343795,  0.02467195, -0.62787765])
    # beta_fama_mom = array([-0.04944137,  0.0455642 , -0.03081603,  1.560502  ])
    
    fig, ax = plt.subplots(figsize = (12, 6))
    zero_cost_returns.plot(ax = ax)
    ax.set_ylabel('returns')


# =============================================================================
# (c)
# =============================================================================
    
    if not LOAD_CACHE:
        
        # get lagged market capitalization of all stocks in one dataframe
        mcap_lag_stocks = pd.DataFrame()
        for permno, stock in stocks.items():
                temp = stock.df[['mcap']].shift(1).rename(columns = {'mcap': permno})
                mcap_lag_stocks = pd.concat([mcap_lag_stocks, temp], axis = 1)
                
        mcap_lag_stocks.to_csv('mcap_lag_stocks.csv')
                
    else: # faster if read from csv
        mcap_lag_stocks = pd.read_csv('mcap_lag_stocks.csv', index_col = 'date')
        mcap_lag_stocks.index = pd.to_datetime(mcap_lag_stocks.index)
        mcap_lag_stocks.columns = mcap_lag_stocks.columns.astype(int)
        
    # drop months where there is nothng (due to shift method)
    mcap_lag_stocks = mcap_lag_stocks.dropna(how = 'all', axis = 0)  
    
    # in each month, sort the stocks into 2 deciles based on market capitalization
    # we omit stocks that were not traded when going through months
    # duplicates = False
    mcap_stocks_deciles = mcap_lag_stocks.apply(lambda x: 1 + pd.qcut(x, 
                                                                      2, 
                                                                      labels = False,
                                                                      duplicates = 'drop'), 
                                                axis = 1) 
    
    # create value-weighted returns every month based on 2 deciles
    vw_returns_bis = pd.DataFrame(index = mcap_stocks_deciles.index,
                                  columns = list(range(1, 3)))
    
    # fill vw_returns according to mcap binning
    # go over all 2 bins
    for b in vw_returns_bis.columns:
        returns_b = ret_stocks.mask(mcap_stocks_deciles != float(b), np.nan)
        mcaps_b = mcap_stocks.mask(mcap_stocks_deciles != float(b), np.nan)
        returns_b = returns_b.dropna(how = 'all')
        mcaps_b = mcaps_b.dropna(how = 'all')        
        weights_b = mcaps_b.apply(lambda row: row/row.sum(), axis = 1)
        vw_returns_b = pd.DataFrame((weights_b*returns_b)\
                                    .sum(axis = 1)/weights_b.sum(axis = 1))
        vw_returns_bis[b] = vw_returns_b

    # Compute the returns on a zero-cost portfolio that 
    # goes long in the group with the highest past returns 
    # and short in the group with the lowest past returns
    zero_cost_returns_bis = - vw_returns_bis[2] + vw_returns_bis[1]
    zero_cost_returns_bis = zero_cost_returns_bis.dropna(how = 'all', axis = 0)
    
    # do a linear regression
    regr_fama = LinearRegression()
    regr_fama = regr_fama.fit(smb_hml, zero_cost_returns_bis)
    alpha_fama_bis = regr_fama.intercept_
    beta_fama_bis = regr_fama.coef_
    
    regr_fama_mom = LinearRegression()
    regr_fama_mom = regr_fama_mom.fit(smb_hml_mom, zero_cost_returns_bis)
    alpha_fama_mom_bis = regr_fama_mom.intercept_
    beta_fama_mom_bis = regr_fama_mom.coef_
    
    # alpha_fama_bis = 0.00960077667829191
    # alpha_fama_mom_bis = 0.011508877741353155
    # beta_fama_bis = array([0.13404236, 1.1668894 , 0.30822139])
    # beta_fama_mom_bis = array([ 0.08727781,  1.16396417,  0.22462377, -0.2184938 ])
    
    fig, ax = plt.subplots(figsize = (12, 6))
    zero_cost_returns_bis.plot(ax = ax)
    ax.set_ylabel('returns')
    